# Counterfactuals benchmark on tabular datasets

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
BASE_PATH = "/gdrive/My Drive/counterfactuals"

Mounted at /gdrive


## Imports and preprocessing

In [ ]:
# Install the dev version of the Alibi package
!pip install git+https://github.com/SeldonIO/alibi.git > /dev/null
from alibi import __version__ as alibi_version
print(f"Alibi version: {alibi_version}")
import logging

alibi_logger = logging.getLogger("alibi")
alibi_logger.setLevel("CRITICAL")

import tensorflow as tf
# Disabling eager execution because Alibi is not compatible with it
tf.compat.v1.disable_eager_execution()
print(f"Is TensorFlow running in eager execution mode? -----→ {tf.executing_eagerly()}")
!nvidia-smi -L

  Running command git clone -q https://github.com/SeldonIO/alibi.git /tmp/pip-req-build-a1d2dpc5
Alibi version: 0.5.6dev
Is TensorFlow running in eager execution mode? -----→ False
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
import os
from datetime import datetime

if not os.path.exists(BASE_PATH):
    os.makedirs(BASE_PATH)
os.chdir(BASE_PATH)

date = datetime.now().strftime('%Y-%m-%d')
EXPERIMENT_PATH = f"{BASE_PATH}/diabetes_{date}"
MODELS_EXPERIMENT_PATH = f"{BASE_PATH}/diabetes_2020-09-09"
if not os.path.exists(EXPERIMENT_PATH):
    os.makedirs(EXPERIMENT_PATH)

## Data import and preprocessing

In [ ]:
import json
import pickle
import time
from matplotlib import offsetbox
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pprint
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


INITIAL_CLASS = 0
DESIRED_CLASS = 1
N_CLASSES = 2

np.set_printoptions(precision=2)
tf.random.set_seed(2020)
np.random.seed(2020)

# Pima indians Diabetes dataset
# https://www.kaggle.com/uciml/pima-indians-diabetes-database
df = pd.read_csv(f"{BASE_PATH}/diabetes.csv", index_col=False)
target_column = "Outcome"
immutable_features = {"Pregnancies", "DiabetesPedigreeFunction", "Age"}

features = set(df.columns) - {target_column}
mutable_features = features - immutable_features
features = list(mutable_features) + list(immutable_features)

x = df[features]
y = df[target_column].values

X_train, X_test, y_train, y_test = train_test_split(df[features].values, y, test_size=0.2)

standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.transform(X_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

df[features].sample(5)

,SkinThickness,BMI,Glucose,Insulin,BloodPressure,Age,Pregnancies,DiabetesPedigreeFunction
379,39,43.4,93,72,100,35,0,1.021
741,20,30.8,102,94,44,26,3,0.400
102,0,22.5,125,0,96,21,0,0.262
641,0,34.3,128,0,70,24,4,0.303
14,19,25.8,166,175,72,51,5,0.587


In [ ]:
def compute_reconstruction_error(x, autoencoder):
    """Compute the reconstruction error for a given autoencoder and data points."""
    preds = autoencoder.predict(x)
    preds_flat = preds.reshape((preds.shape[0], -1))
    x_flat = x.reshape((x.shape[0], -1))
    return np.linalg.norm(x_flat - preds_flat, axis=1)

def format_metric(metric):
    """Return a formatted version of a metric, with the confidence interval."""
    return f"{metric.mean():.3f} ± {1.96*metric.std()/np.sqrt(len(metric)):.3f}"

def compute_metrics(samples, counterfactuals, latencies, classifier, autoencoder,
                    batch_latency=None):
    """ Summarize the relevant metrics in a dictionary. """
    reconstruction_error = compute_reconstruction_error(counterfactuals, autoencoder)
    delta = np.abs(samples-counterfactuals)
    l1_distances = delta.reshape(delta.shape[0], -1).sum(axis=1)
    prediction_gain = (
        classifier.predict(counterfactuals)[:, DESIRED_CLASS] - 
        classifier.predict(samples)[:, DESIRED_CLASS]
    )

    metrics = dict()
    metrics["reconstruction_error"] = format_metric(reconstruction_error)
    metrics["prediction_gain"] = format_metric(prediction_gain)
    metrics["sparsity"] = format_metric(l1_distances)
    metrics["latency"] = format_metric(latencies)
    batch_latency = batch_latency if batch_latency else sum(latencies)
    metrics["latency_batch"] = f"{batch_latency:.3f}"

    return metrics

def save_experiment(method_name, samples, counterfactuals, latencies, 
                    batch_latency=None):
    """Create an experiment folder and save counterfactuals, latencies and metrics."""
    if not os.path.exists(f"{EXPERIMENT_PATH}/{method_name}"):
        os.makedirs(f"{EXPERIMENT_PATH}/{method_name}")   

    np.save(f"{EXPERIMENT_PATH}/{method_name}/counterfactuals.npy", counterfactuals)
    np.save(f"{EXPERIMENT_PATH}/{method_name}/latencies.npy", latencies)

    metrics = compute_metrics(samples, counterfactuals, latencies, classifier, autoencoder)
    json.dump(metrics, open(f"{EXPERIMENT_PATH}/{method_name}/metrics.json", "w"))
    pprint(metrics)

## Train classifier

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Add, Input, ActivityRegularization
from tensorflow.keras import Model, optimizers, regularizers
from tensorflow.keras.utils import to_categorical

tf.random.set_seed(2020)
np.random.seed(2020)

def create_classifier(input_shape):
    """Define and compile a neural network binary classifier.""" 
    model = Sequential([
        Dense(20, activation='relu', input_shape=input_shape),
        Dense(20, activation='relu'),
        Dense(2, activation='softmax'),
    ], name="classifier")
    optimizer = optimizers.Adam(lr=0.0002, beta_1=0.5)
    model.compile(optimizer, 'binary_crossentropy', ['accuracy'])
    return model

classifier = create_classifier((x.shape[1],))

training = classifier.fit(X_train, y_train, batch_size=32, epochs=200, verbose=0,
                          validation_data=(X_test, y_test),)
print(f"Training: loss={training.history['loss'][-1]:.4f}, "
      f"accuracy={training.history['accuracy'][-1]:.4f}")
print(f"Validation: loss={training.history['val_loss'][-1]:.4f}, "
      f"accuracy={training.history['val_accuracy'][-1]:.4f}")

classifier.save(f"{EXPERIMENT_PATH}/classifier.h5")

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


Training: loss=0.4360, accuracy=0.7964
Validation: loss=0.4581, accuracy=0.7727


## Estimate density with the reconstruction error of a (denoising) autoencoder


In [ ]:
def add_noise(x, noise_factor=1e-6):
    x_noisy = x + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x.shape) 
    return x_noisy

    
def create_autoencoder(in_shape=(x.shape[1],)):
    input_ = Input(shape=in_shape) 

    x = Dense(32, activation="relu")(input_)
    encoded = Dense(8)(x)
    x = Dense(32, activation="relu")(encoded)
    decoded = Dense(in_shape[0], activation="tanh")(x)

    autoencoder = Model(input_, decoded)
    optimizer = optimizers.Nadam()
    autoencoder.compile(optimizer, 'mse')
    return autoencoder

autoencoder = create_autoencoder()
training = autoencoder.fit(
    add_noise(X_train), X_train, epochs=100, batch_size=32, shuffle=True, 
    validation_data=(X_test, X_test), verbose=0
)
print(f"Training loss: {training.history['loss'][-1]:.4f}")
print(f"Validation loss: {training.history['val_loss'][-1]:.4f}")

n_samples = 1000
# Compute the reconstruction error of noise data
samples = np.random.randn(n_samples, X_train.shape[1])
reconstruction_error_noise = compute_reconstruction_error(samples, autoencoder)

# Save and print the autoencoder metrics
reconstruction_error = compute_reconstruction_error(X_test, autoencoder)
autoencoder_metrics = {
    "reconstruction_error": format_metric(reconstruction_error),
    "reconstruction_error_noise": format_metric(reconstruction_error_noise),
}
json.dump(autoencoder_metrics, open(f"{EXPERIMENT_PATH}/autoencoder_metrics.json", "w"))
pprint(autoencoder_metrics)

autoencoder.save(f"{EXPERIMENT_PATH}/autoencoder.h5")

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


Training loss: 0.2337
Validation loss: 0.2504
{'reconstruction_error': '1.120 ± 0.137',
 'reconstruction_error_noise': '1.151 ± 0.035'}


## Regularized Gradient Descent

In [ ]:
from alibi.explainers import CounterFactual

shape = (1,) + X_train.shape[1:]
feature_range = (X_train.min(), X_train.max())

cf = CounterFactual(classifier, shape=shape, target_proba=1.0, tol=0.5,
                    target_class=DESIRED_CLASS, max_iter=100, lam_init=0.001,
                    max_lam_steps=5, learning_rate_init=0.1,
                    feature_range=feature_range)

sample = X_test[0]

t_initial = time.time()
explanation = cf.explain(np.expand_dims(sample, axis=0))
print(f"Produced explanation in {time.time() - t_initial:.2f} seconds ")

y_prob = classifier.predict(np.expand_dims(sample, axis=0))[0]
print(f'Original prediction: {y_prob.argmax()} with probability {y_prob.max():.3f}')

pred_class = explanation.cf['class']
proba = explanation.cf['proba'][0][pred_class]
print(f'Counterfactual prediction: {pred_class} with probability {proba:.3f}')

perturbations = (explanation.cf['X'] - sample)[0]
perturbations[-len(immutable_features):] = 0.
print(f"Suggested perturbations: {perturbations}")

Produced explanation in 1.95 seconds 
Original prediction: 0 with probability 0.833
Counterfactual prediction: 1 with probability 0.524
Suggested perturbations: [ 0.35  0.39  0.39  0.39 -0.32  0.    0.    0.  ]


In [ ]:
samples = X_test 

latencies = np.empty(len(samples))
counterfactuals = np.empty_like(samples)

for i, sample in enumerate(samples):
    if ((i % 20) == 0) or (i == (len(samples)-1)):
        print(f"Iteration {i} at {datetime.now()}")
    t_initial = time.time()
    try:
        explanation = cf.explain(np.expand_dims(sample, axis=0))
        counterfactuals[i] = explanation.cf['X']
    except (UnboundLocalError, TypeError):  # counterfactual search failed
        print(f"{i}-th sampled failed")
        counterfactuals[i] = sample
    latencies[i] = 1000*(time.time() - t_initial)

print("Metrics before immutable features projection:")
pprint(compute_metrics(samples, counterfactuals, latencies, classifier, autoencoder,
                    batch_latency=None))
print("-"*80)

# Set immutable features to original values
counterfactuals[:, len(mutable_features):] = samples[:, len(mutable_features):]

print("Metrics after immutable features projection:")
save_experiment("rgd", samples, counterfactuals, latencies)

1-th iteration at 2020-09-17 04:57:43.519545
21-th iteration at 2020-09-17 04:58:11.216447
41-th iteration at 2020-09-17 04:58:37.325165
61-th iteration at 2020-09-17 04:59:05.829140
81-th iteration at 2020-09-17 04:59:32.625520
101-th iteration at 2020-09-17 04:59:57.687546
121-th iteration at 2020-09-17 05:00:25.043526
141-th iteration at 2020-09-17 05:00:50.304196
154-th iteration at 2020-09-17 05:01:06.863386
Metrics before immutable features projection:
{'latency': '1328.412 ± 14.921',
 'latency_batch': '204575.454',
 'prediction_gain': '0.323 ± 0.048',
 'reconstruction_error': '1.618 ± 0.204',
 'sparsity': '2.602 ± 0.311'}
--------------------------------------------------------------------------------
Metrics after immutable features projection:
{'latency': '1328.412 ± 14.921',
 'latency_batch': '204575.454',
 'prediction_gain': '0.266 ± 0.049',
 'reconstruction_error': '1.582 ± 0.205',
 'sparsity': '1.968 ± 0.300'}


## Counterfactual Search Guided by Prototypes

In [ ]:
from alibi.explainers import CounterFactualProto

shape = (1,) + X_train.shape[1:]
feature_range = (X_train.min(), X_train.max())

cf_proto = CounterFactualProto(
    classifier, shape, use_kdtree=True, theta=10., feature_range=feature_range,
    max_iterations=200, c_steps=10
)
cf_proto.fit(X_train, trustscore_kwargs=None);

In [ ]:
sample = X_test[0]


t_initial = time.time()
explanation = cf_proto.explain(
    np.expand_dims(sample, axis=0), k=5, k_type='mean', target_class=[DESIRED_CLASS]
)

print(f"Produced explanation in {time.time() - t_initial:.2f} seconds ")

y_prob = classifier.predict(np.expand_dims(sample, axis=0))[0]
print(f'Original prediction: {y_prob.argmax()} with probability {y_prob.max():.3f}')

pred_class = explanation.cf['class']
proba = explanation.cf['proba'][0][pred_class]
print(f'Counterfactual prediction: {pred_class} with probability {proba:.3f}')

perturbations = (explanation.cf['X'] - sample)[0]
perturbations[-len(immutable_features):] = 0.
print(f"Suggested perturbations: {perturbations}")

Produced explanation in 3.55 seconds 
Original prediction: 0 with probability 0.833
Counterfactual prediction: 1 with probability 0.506
Suggested perturbations: [-5.46e-08 -3.65e-10  3.04e-01 -2.61e-08 -6.28e-10  0.00e+00  0.00e+00
  0.00e+00]


In [ ]:
verbose = False
samples = X_test

latencies = np.empty(len(samples))
counterfactuals = np.empty_like(samples)
for i, sample in enumerate(samples):
    if ((i % 20) == 0) or (i == (len(samples)-1)):
        print(f"{i+1}-th iteration at {datetime.now()}")
    t_initial = time.time()
    try:
        explanation = cf_proto.explain(np.expand_dims(sample, axis=0), k=20, 
                                       k_type='mean', target_class=[DESIRED_CLASS])
        counterfactuals[i] = explanation.cf['X']
    except (UnboundLocalError, TypeError) as e:  # counterfactual search failed
        if verbose:
            print(f"{i}-th sampled failed")
        counterfactuals[i] = sample
    latencies[i] = 1000*(time.time() - t_initial)

print("Metrics before immutable features projection:")
pprint(compute_metrics(samples, counterfactuals, latencies, classifier, autoencoder,
                    batch_latency=None))
print("-"*80)

# Set immutable features to original values
counterfactuals[:, len(mutable_features):] = samples[:, len(mutable_features):]

print("Metrics after immutable features projection:")
save_experiment("csgp", samples, counterfactuals, latencies)

1-th iteration at 2020-09-17 05:01:15.294025
21-th iteration at 2020-09-17 05:02:16.691610
41-th iteration at 2020-09-17 05:03:18.540155
61-th iteration at 2020-09-17 05:04:21.173208
81-th iteration at 2020-09-17 05:05:24.615457
101-th iteration at 2020-09-17 05:06:28.014770
121-th iteration at 2020-09-17 05:07:31.594086
141-th iteration at 2020-09-17 05:08:34.903332
154-th iteration at 2020-09-17 05:09:16.161087
Metrics before immutable features projection:
{'latency': '3142.089 ± 13.944',
 'latency_batch': '483881.636',
 'prediction_gain': '0.212 ± 0.030',
 'reconstruction_error': '0.915 ± 0.104',
 'sparsity': '1.696 ± 0.269'}
--------------------------------------------------------------------------------
Metrics after immutable features projection:
{'latency': '3142.089 ± 13.944',
 'latency_batch': '483881.636',
 'prediction_gain': '0.147 ± 0.024',
 'reconstruction_error': '0.961 ± 0.111',
 'sparsity': '1.198 ± 0.200'}


## GAN-based counterfactual search

In [ ]:
def generate_fake_samples(x, generator):
    """Use the input generator to generate samples."""
    return generator.predict(x)

def data_stream(x, y=None, batch_size=500):
    """Generate batches until exhaustion of the input data."""
    n_train = x.shape[0]
    if y is not None:
        assert n_train == len(y)
    n_complete_batches, leftover = divmod(n_train, batch_size)
    n_batches = n_complete_batches + bool(leftover)

    perm = np.random.permutation(n_train)
    for i in range(n_batches):
        batch_idx = perm[i * batch_size:(i + 1) * batch_size]
        if y is not None:
            output = (x[batch_idx], y[batch_idx])
        else:
            output = x[batch_idx]
        yield output


def infinite_data_stream(x, y=None, batch_size=500):
    """Infinite batch generator."""
    batches = data_stream(x, y, batch_size=batch_size)
    while True:
        try:
            yield next(batches)
        except StopIteration:
            batches = data_stream(x, y, batch_size=batch_size)
            yield next(batches)

def create_generator(in_shape=(x_train.shape[1],), residuals=True):
    """Define and compile the residual generator of the CounteRGAN."""
    generator_input = Input(shape=in_shape, name='generator_input')
    generator = Dense(64, activation='relu')(generator_input)
    generator = Dense(32, activation='relu')(generator)
    generator = Dense(64, activation='relu')(generator)
    generator = Dense(in_shape[0], activation='tanh')(generator)
    generator_output = ActivityRegularization(l1=0., l2=1e-6)(generator)
    
    if residuals:
        generator_output = Add(name="output")([generator_input, generator_output])

    return Model(inputs=generator_input, outputs=generator_output)


def create_discriminator(in_shape=(X_train.shape[1],)):
    """ Define a neural network binary classifier to classify real and generated 
    examples."""
    model = Sequential([
        Dense(128, activation='relu', input_shape=in_shape),
        Dropout(0.2),
        Dense(1, activation='sigmoid'),
    ], name="discriminator")
    optimizer = optimizers.Adam(lr=0.0005, beta_1=0.5, decay=1e-8)
    model.compile(optimizer, 'binary_crossentropy', ['accuracy'])
    return model


def define_countergan(generator, discriminator, classifier, 
                      input_shape=(X_train.shape[1],)):
    """Combine a generator, discriminator, and fixed classifier into the CounteRGAN."""
    discriminator.trainable = False
    classifier.trainable = False

    countergan_input = Input(shape=input_shape, name='countergan_input')
  
    x_generated = generator(countergan_input)

    countergan = Model(
        inputs=countergan_input, 
        outputs=[discriminator(x_generated), classifier(x_generated)]
    )
        
    optimizer = optimizers.RMSprop(lr=2e-4, decay=1e-8)
    countergan.compile(optimizer, ["binary_crossentropy", "categorical_crossentropy"])
    return countergan


def define_weighted_countergan(generator, discriminator, 
                               input_shape=(X_train.shape[1],)):
    """Combine a generator and a discriminator for the weighted version of the 
    CounteRGAN."""
    discriminator.trainable = False
    classifier.trainable = False
    countergan_input = Input(shape=input_shape, name='countergan_input')
  
    x_generated = generator(countergan_input)

    countergan = Model(inputs=countergan_input, outputs=discriminator(x_generated))
    optimizer = optimizers.RMSprop(lr=5e-4, decay=1e-8)
    countergan.compile(optimizer, "binary_crossentropy")  
    return countergan



In [ ]:
def train_countergan(n_discriminator_steps, n_generator_steps, n_training_iterations,
                     classifier, discriminator, generator, batches, 
                     weighted_version=False):
    """ Main function: train the CounteRGAN"""
    def check_divergence(x_generated):
        return np.all(np.isnan(x_generated))

    def print_training_information(generator, classifier, X_test, iteration):
        X_gen = generator.predict(X_test)
        clf_pred_test = classifier.predict(X_test)
        clf_pred = classifier.predict(X_gen)

        delta_clf_pred = (clf_pred - clf_pred_test)[:, DESIRED_CLASS]
        y_target = to_categorical([DESIRED_CLASS] * len(clf_pred), 
                                  num_classes=N_CLASSES)
        print('='*88)
        print(f"Training iteration {iteration} at {datetime.now()}")
        
        
        reconstruction_error = np.mean(compute_reconstruction_error(X_gen, autoencoder))
        print(f"Autoencoder reconstruction error (infinity to 0): {reconstruction_error:.3f}")
        print(f"Counterfactual prediction gain (0 to 1): {delta_clf_pred.mean():.3f}")
        print(f"Sparsity (L1, infinity to 0): {np.mean(np.abs(X_gen-X_test)):.3f}")

    if weighted_version:
        countergan = define_weighted_countergan(generator, discriminator)
    else:
        countergan = define_countergan(generator, discriminator, classifier)

    for iteration in range(n_training_iterations):
        if iteration > 0:
            x_generated = generator.predict(x_fake_input)
            if check_divergence(x_generated):
                print("Training diverged with the following loss functions:")
                print(discrim_loss_1, discrim_accuracy, gan_loss, 
                    discrim_loss, discrim_loss_2, clf_loss)
                break

        # Periodically print and plot training information 
        if (iteration % 1000 == 0) or (iteration == n_training_iterations - 1):
            print_training_information(generator, classifier, X_test, iteration)

        # Train the discriminator
        discriminator.trainable = True
        for _ in range(n_discriminator_steps):
            x_fake_input, _ = next(batches)
            x_fake = generate_fake_samples(x_fake_input, generator)
            x_real = x_fake_input

            x_batch = np.concatenate([x_real, x_fake])
            y_batch = np.concatenate([np.ones(len(x_real)), np.zeros(len(x_fake))])
            
            # Shuffle real and fake examples
            p = np.random.permutation(len(y_batch))
            x_batch, y_batch = x_batch[p], y_batch[p]

            if weighted_version:
                classifier_scores = classifier.predict(x_batch)[:, DESIRED_CLASS]
                
                # The following update to the classifier scores is needed to have the 
                # same order of magnitude between real and generated samples losses
                real_samples = np.where(y_batch == 1.)
                average_score_real_samples = np.mean(classifier_scores[real_samples])
                classifier_scores[real_samples] /= average_score_real_samples
                
                fake_samples = np.where(y_batch == 0.)
                classifier_scores[fake_samples] = 1.

                discriminator.train_on_batch(
                    x_batch, y_batch, sample_weight=classifier_scores
                )
            else:
                discriminator.train_on_batch(x_batch, y_batch)

        # Train the generator 
        discriminator.trainable = False
        for _ in range(n_generator_steps):
            x_fake_input, _ = next(batches)
            y_fake = np.ones(len(x_fake_input))
            if weighted_version:
                countergan.train_on_batch(x_fake_input, y_fake)
            else:
                y_target = to_categorical([DESIRED_CLASS] * len(x_fake_input), 
                                          num_classes=N_CLASSES)
                countergan.train_on_batch(x_fake_input, [y_fake, y_target])
    return countergan

## Counterfactual search with a regular GAN

In [ ]:
discriminator = create_discriminator()
generator = create_generator(residuals=False)
batches = infinite_data_stream(X_train, y_train, batch_size=256)

method_name = "regular_gan"
countergan = train_countergan(2, 4, 2000, classifier, discriminator, generator, batches)

t_initial = time.time()
counterfactuals = generator.predict(X_test)
batch_latency = 1000*(time.time() - t_initial)

latencies = np.zeros(len(X_test))
for i, x in enumerate(X_test):
    t_initial = time.time()
    _ = generator.predict(np.expand_dims(x, axis=0))
    latencies[i] = 1000*(time.time() - t_initial)

print("-"*80)
print("Metrics before immutable features projection:")
pprint(compute_metrics(samples, counterfactuals, latencies, classifier, autoencoder,
                    batch_latency=None))
print("-"*80)

# Set immutable features to original values
counterfactuals[:, len(mutable_features):] = samples[:, len(mutable_features):]

print("Metrics after immutable features projection:")
save_experiment(method_name, X_test, counterfactuals, latencies, batch_latency)

generator.save(f"{EXPERIMENT_PATH}/{method_name}/generator.h5", save_format='h5')
discriminator.save(f"{EXPERIMENT_PATH}/{method_name}/discriminator.h5", save_format='h5')
countergan.save(f"{EXPERIMENT_PATH}/{method_name}/countergan.h5", save_format='h5')

Training iteration 0 at 2020-09-17 05:29:44.184846
Autoencoder reconstruction error (infinity to 0): 0.401
Counterfactual prediction gain (0 to 1): 0.069
Sparsity (L1, infinity to 0): 0.777
Training iteration 1000 at 2020-09-17 05:30:53.071830
Autoencoder reconstruction error (infinity to 0): 0.384
Counterfactual prediction gain (0 to 1): -0.045
Sparsity (L1, infinity to 0): 1.110
Training iteration 1999 at 2020-09-17 05:31:52.525108
Autoencoder reconstruction error (infinity to 0): 0.499
Counterfactual prediction gain (0 to 1): -0.059
Sparsity (L1, infinity to 0): 1.074
--------------------------------------------------------------------------------
Metrics before immutable features projection:
{'latency': '1.681 ± 0.056',
 'latency_batch': '258.897',
 'prediction_gain': '-0.051 ± 0.052',
 'reconstruction_error': '0.514 ± 0.012',
 'sparsity': '8.623 ± 0.383'}
--------------------------------------------------------------------------------
Metrics after immutable features projection:
{

## CounteRGAN: first formulation for differentiable classifiers

In [ ]:
discriminator = create_discriminator()
generator = create_generator(residuals=True)
batches = infinite_data_stream(X_train, y_train, batch_size=256)

method_name = "countergan"
countergan = train_countergan(2, 4, 2000, classifier, discriminator, generator, batches)

t_initial = time.time()
counterfactuals = generator.predict(X_test)
batch_latency = 1000*(time.time() - t_initial)

latencies = np.zeros(len(X_test))
for i, x in enumerate(X_test):
    t_initial = time.time()
    _ = generator.predict(np.expand_dims(x, axis=0))
    latencies[i] = 1000*(time.time() - t_initial)

print("-"*80)
print("Metrics before immutable features projection:")
pprint(compute_metrics(samples, counterfactuals, latencies, classifier, autoencoder,
                    batch_latency=None))
print("-"*80)

# Set immutable features to original values
counterfactuals[:, len(mutable_features):] = samples[:, len(mutable_features):]

print("Metrics after immutable features projection:")
save_experiment(method_name, X_test, counterfactuals, latencies, batch_latency)

generator.save(f"{EXPERIMENT_PATH}/{method_name}/generator.h5", save_format='h5')
discriminator.save(f"{EXPERIMENT_PATH}/{method_name}/discriminator.h5", save_format='h5')
countergan.save(f"{EXPERIMENT_PATH}/{method_name}/countergan.h5", save_format='h5')

Training iteration 0 at 2020-09-17 05:27:28.122947
Autoencoder reconstruction error (infinity to 0): 1.105
Counterfactual prediction gain (0 to 1): -0.039
Sparsity (L1, infinity to 0): 0.115
Training iteration 1000 at 2020-09-17 05:28:37.563724
Autoencoder reconstruction error (infinity to 0): 1.383
Counterfactual prediction gain (0 to 1): 0.260
Sparsity (L1, infinity to 0): 0.481
Training iteration 1999 at 2020-09-17 05:29:35.901126
Autoencoder reconstruction error (infinity to 0): 1.377
Counterfactual prediction gain (0 to 1): 0.254
Sparsity (L1, infinity to 0): 0.484
--------------------------------------------------------------------------------
Metrics before immutable features projection:
{'latency': '1.506 ± 0.028',
 'latency_batch': '231.963',
 'prediction_gain': '0.253 ± 0.031',
 'reconstruction_error': '1.378 ± 0.156',
 'sparsity': '3.862 ± 0.158'}
--------------------------------------------------------------------------------
Metrics after immutable features projection:
{'l

## CounteRGAN: second formulation for any classifier

In [ ]:
discriminator = create_discriminator()
generator = create_generator(residuals=True)
batches = infinite_data_stream(X_train, y_train, batch_size=256)

method_name = "countergan-wt"
countergan = train_countergan(2, 3, 2000, classifier, discriminator, generator, 
                              batches, weighted_version=True)

t_initial = time.time()
counterfactuals = generator.predict(X_test)
batch_latency = 1000*(time.time() - t_initial)

latencies = np.zeros(len(X_test))
for i, x in enumerate(X_test):
    t_initial = time.time()
    _ = countergan.predict(np.expand_dims(x, axis=0))
    latencies[i] = 1000*(time.time() - t_initial)

print("-"*80)
print("Metrics before immutable features projection:")
pprint(compute_metrics(samples, counterfactuals, latencies, classifier, autoencoder,
                    batch_latency=None))
print("-"*80)

# Set immutable features to original values
counterfactuals[:, len(mutable_features):] = samples[:, len(mutable_features):]

print("Metrics after immutable features projection:")
save_experiment(method_name, X_test, counterfactuals, latencies, batch_latency)

generator.save(f"{EXPERIMENT_PATH}/{method_name}/generator.h5", save_format='h5')
discriminator.save(f"{EXPERIMENT_PATH}/{method_name}/discriminator.h5", save_format='h5')
countergan.save(f"{EXPERIMENT_PATH}/{method_name}/countergan.h5", save_format='h5')

Training iteration 0 at 2020-09-17 05:55:16.474251
Autoencoder reconstruction error (infinity to 0): 1.179
Counterfactual prediction gain (0 to 1): 0.021
Sparsity (L1, infinity to 0): 0.121
Training iteration 1000 at 2020-09-17 05:56:43.640654
Autoencoder reconstruction error (infinity to 0): 1.365
Counterfactual prediction gain (0 to 1): 0.170
Sparsity (L1, infinity to 0): 0.424
Training iteration 1999 at 2020-09-17 05:58:05.180140
Autoencoder reconstruction error (infinity to 0): 1.367
Counterfactual prediction gain (0 to 1): 0.212
Sparsity (L1, infinity to 0): 0.439
--------------------------------------------------------------------------------
Metrics before immutable features projection:
{'latency': '10.295 ± 16.699',
 'latency_batch': '1585.397',
 'prediction_gain': '0.225 ± 0.027',
 'reconstruction_error': '1.373 ± 0.152',
 'sparsity': '3.541 ± 0.188'}
--------------------------------------------------------------------------------
Metrics after immutable features projection:
{

## Generate the benchmark table

In [ ]:
METHODS = ["rgd", "csgp", "regular_gan", "countergan", "countergan-wt"]
METRIC_NAMES = [
    "prediction_gain", "reconstruction_error", "sparsity", "latency", "latency_batch"
]

metrics = dict()
for method in METHODS:
    method_metrics = json.load(open(f"{EXPERIMENT_PATH}/{method}/metrics.json", "r"))
    method_metrics = {k: v for k, v in method_metrics.items() if k in METRIC_NAMES}
    metrics[method] = method_metrics

metrics = pd.DataFrame(metrics)
metrics.columns =  ["RGD",  "CSGP", "GAN", "CounterGAN", "CounterRGAN-wt"] 

metrics.index = [
    "↓ Realism",
    "↑ Prediction gain",
    "↓ Sparsity",
    "↓ Latency (ms)",
    "↓ Batch latency (ms)",
]

metrics

## Individual examples

In [ ]:
negative_idx = np.where(classifier.predict(x_test)[:, 1] < 0.5)[0]
x_negative = X_test[negative_idx]
original_features = standard_scaler.inverse_transform(x_negative)
negative_df = pd.DataFrame(original_features, columns=features)
negative_df.head(5)

,BloodPressure,Insulin,Glucose,BMI,SkinThickness,DiabetesPedigreeFunction,Age,Pregnancies
0,76.0,0.0,115.0,31.2,0.0,0.343,44.0,5.000000e+00
1,56.0,0.0,122.0,33.3,0.0,1.114,33.0,9.000000e+00
2,72.0,0.0,118.0,23.1,19.0,1.476,46.0,8.000000e+00
3,64.0,66.0,84.0,35.8,22.0,0.545,21.0,4.440892e-16
4,82.0,125.0,84.0,38.2,31.0,0.233,23.0,4.440892e-16


In [ ]:
counterfactuals = standard_scaler.inverse_transform(
    generator.predict(x_test[negative_idx])
)
residuals = (counterfactuals - 
             standard_scaler.inverse_transform(x_test[negative_idx]))
residuals_df = pd.DataFrame(residuals, columns=features)
residuals_df[list(immutable_features)] = 0.
residuals_df.head(5)

,BloodPressure,Insulin,Glucose,BMI,SkinThickness,DiabetesPedigreeFunction,Age,Pregnancies
0,1.245267,-0.168285,29.517547,0.265268,2.824298,0.0,0.0,0.0
1,11.590098,-10.985187,-4.899478,4.001734,-4.969263,0.0,0.0,0.0
2,16.391856,-0.233478,-0.011665,-1.376831,3.959885,0.0,0.0,0.0
3,-2.812314,29.489317,7.159651,5.620124,9.216398,0.0,0.0,0.0
4,-8.306026,60.743688,26.160375,5.256007,9.130106,0.0,0.0,0.0


In [ ]:
sample_idx = 20
sample = np.expand_dims(X_test[sample_idx], axis=0)

def compute_residuals(sample, counterfactual):
    counterfactual = standard_scaler.inverse_transform(counterfactual)
    residuals = (counterfactual - standard_scaler.inverse_transform(sample))[0]
    residuals[-len(immutable_features):] = 0
    return residuals

method_outputs = dict()

d = negative_df.iloc[sample_idx].to_dict()
d["Classifier Prediction"] = classifier.predict(sample)[0][1]
method_outputs["Initial values"] = d


explanation = cf.explain(sample)
counterfactual = explanation.cf['X']
scaled_counterfactual = compute_residuals(sample, counterfactual)
d = {k: v for k, v in zip(features, list(scaled_counterfactual))}
d["Classifier Prediction"] = classifier.predict(counterfactual)[0][1]
method_outputs["RGD"] = d

explanation = cf_proto.explain(sample, k=5, k_type='mean', target_class=[DESIRED_CLASS])
counterfactual = explanation.cf['X']
scaled_counterfactual = compute_residuals(sample, counterfactual)
d = {k: v for k, v in zip(features, list(scaled_counterfactual))}
d["Classifier Prediction"] = classifier.predict(counterfactual)[0][1]
method_outputs["CSGP"] = d

for method in ["regular_gan", "countergan", "countergan-wt"]:
    generator = load_model(f"{EXPERIMENT_PATH}/{method}/generator.h5")
    counterfactual = generator.predict(sample)
    scaled_counterfactual = compute_residuals(sample, counterfactual)
    d = {k: v for k, v in zip(features, list(scaled_counterfactual))}
    d["Classifier Prediction"] = classifier.predict(counterfactual)[0][1]
    method_outputs[method] = d

df = pd.DataFrame(method_outputs)
df

regular_gan
countergan
countergan-wt


,Initial values,RGD,CSGP,regular_gan,countergan,countergan-wt
BloodPressure,78.000000,-5.423836,0.000000,-5.299530,-12.046440,15.693680
Insulin,22.000000,-18.605087,0.000000,85.835007,-50.358517,26.346497
Glucose,126.000000,9.543411,12.605164,-33.490791,30.281281,25.446625
BMI,29.600000,2.230800,0.754640,8.812698,-0.522469,-5.058810
SkinThickness,27.000000,-1.940363,0.000000,18.851517,3.495853,8.009716
DiabetesPedigreeFunction,0.439000,0.000000,0.000000,0.000000,0.000000,0.000000
Age,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Pregnancies,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Classifier Prediction,0.185442,0.576988,0.501705,0.401092,0.623442,0.585553
